In [32]:
import tensorflow as tf

from tensorflow.image import resize
from tensorflow.keras.backend import clear_session
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.metrics import  Recall, CategoricalAccuracy
from IPython.display import clear_output
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import concatenate as concat
from scipy.stats import entropy
import os

from imblearn.under_sampling import RandomUnderSampler

from helpers.help import *
from helpers.helptf import *
from sklearn.utils import resample

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input

from PIL import Image

In [33]:
# load dataset
df = pd.read_csv('hr-dataset/full_df.csv')

#Left eye
# get the diagnostic of hypertensive retinopathy
ds_hr_left = df[df['Left-Diagnostic Keywords'].str.contains('hypertensive retinopathy', na=False)]
# get the diagnostic of diabetic retinopathy
ds_dr_left = df[df['Left-Diagnostic Keywords'].str.contains('diabetic retinopathy', na=False)]
# get the diagnostic of normal fundus
ds_normal_left = df[df['Left-Diagnostic Keywords'] == 'normal fundus']


#Right eye
# get the diagnostic of hypertensive retinopathy
ds_hr_right = df[df['Right-Diagnostic Keywords'].str.contains('hypertensive retinopathy', na=False)]
# get the diagnostic of diabetic retinopathy
ds_dr_right = df[df['Right-Diagnostic Keywords'].str.contains('diabetic retinopathy', na=False)]
# get the diagnostic of normal fundus
ds_normal_right = df[df['Right-Diagnostic Keywords'] == 'normal fundus']


# Specific dataframe
# Left eye
df_hr_left = ds_hr_left[['Left-Diagnostic Keywords', 'Left-Fundus']]
df_dr_left = ds_dr_left[['Left-Diagnostic Keywords', 'Left-Fundus']]
df_normal_left = ds_normal_left[['Left-Diagnostic Keywords', 'Left-Fundus']]

# Right eye
df_hr_right = ds_hr_right[['Right-Diagnostic Keywords', 'Right-Fundus']]
df_dr_right = ds_dr_right[['Right-Diagnostic Keywords', 'Right-Fundus']]
df_normal_right = ds_normal_right[['Right-Diagnostic Keywords', 'Right-Fundus']]


# Droping class
# Left eye
df_hr_left = df_hr_left.drop('Left-Diagnostic Keywords', axis=1)
df_dr_left = df_dr_left.drop('Left-Diagnostic Keywords', axis=1)
df_normal_left = df_normal_left.drop('Left-Diagnostic Keywords', axis=1)
# Right eye
df_hr_right = df_hr_right.drop('Right-Diagnostic Keywords', axis=1)
df_dr_right = df_dr_right.drop('Right-Diagnostic Keywords', axis=1)
df_normal_right = df_normal_right.drop('Right-Diagnostic Keywords', axis=1)



print("---- LEFT ----")
print(df_hr_left.shape[0])
print(df_dr_left.shape[0])
print(df_normal_left.shape[0])
print("---- RIGHT ----")
print(df_hr_right.shape[0])
print(df_dr_right.shape[0])
print(df_normal_right.shape[0])

# join dataframes
df_hr = pd.concat([df_hr_left, df_hr_right])
df_dr = pd.concat([df_dr_left, df_dr_right])
df_normal = pd.concat([df_normal_left, df_normal_right])


---- LEFT ----
191
85
2796
---- RIGHT ----
191
80
2705


# Variables preparation

In [ ]:
path = os.path.join(os.getcwd(),'hr-dataset/preprocessed_images')

pathHr = os.path.join(os.getcwd(),'hr-dataset/augmented_images/hr')

pathDr = os.path.join(os.getcwd(),'hr-dataset/augmented_images/dr')

detailPath = os.path.join(path,'11_left.jpg')

data_augmentation = keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.4),
])

# Hypertensive Retinopathy augmentation

In [35]:
for index, row in df_hr.iterrows():
    if type(row['Left-Fundus']) != float:
        detailPath = os.path.join(path,row['Left-Fundus'])
    else:
        detailPath = os.path.join(path,row['Right-Fundus'])
    if(os.path.exists(detailPath)):
        image_readed = tf.io.read_file(detailPath)
        image_readed = tf.image.decode_image(image_readed,channels=3,expand_animations = False)
        for i in range(6):
          augmented_image = data_augmentation(image_readed)
          augmented_image_np = augmented_image.numpy().astype('uint8')

          # Convert NumPy array to image
          image = Image.fromarray(augmented_image_np)

          # Save image
          if type(row['Left-Fundus']) != float:
            image.save(pathHr + "/" + row['Left-Fundus'].split('.')[0] + "_" + str(i) +".jpg")
          else:
            image.save(pathHr + "/" + row['Right-Fundus'].split('.')[0] + "_" + str(i) +".jpg")


# Diabetic Retinopathy augmentation

In [36]:
for index, row in df_dr.iterrows():
    if type(row['Left-Fundus']) != float:
        detailPath = os.path.join(path,row['Left-Fundus'])
    else:
        detailPath = os.path.join(path,row['Right-Fundus'])
    if(os.path.exists(detailPath)):
        image_readed = tf.io.read_file(detailPath)
        image_readed = tf.image.decode_image(image_readed,channels=3,expand_animations = False)
        for i in range(6):
          augmented_image = data_augmentation(image_readed)
          augmented_image_np = augmented_image.numpy().astype('uint8')

          # Convert NumPy array to image
          image = Image.fromarray(augmented_image_np)

          # Save image
          if type(row['Left-Fundus']) != float:
            image.save(pathDr + "/" + row['Left-Fundus'].split('.')[0] + "_" + str(i) +".jpg")
          else:
            image.save(pathDr + "/" + row['Right-Fundus'].split('.')[0] + "_" + str(i) +".jpg")